# Gerekli importlar yapıldı

In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE

# Dosya okunuyor

In [2]:
df=pd.read_csv('failure.csv')

# EDA 

In [10]:
df.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [4]:
df.isnull().sum()

date          0
device        0
failure       0
attribute1    0
attribute2    0
attribute3    0
attribute4    0
attribute5    0
attribute6    0
attribute7    0
attribute8    0
attribute9    0
dtype: int64

In [5]:
df.shape

(124494, 12)

# Dosyamızdaki dengesiz verileri smote ile düzenliyoruz

In [5]:
# Özellikleri ve hedef değişkeni ayır
x = df.drop(columns=['failure', 'date', 'device'])
y = df['failure']

# SMOTE uygulayarak veriyi dengele
oversample = SMOTE()
x, y = oversample.fit_resample(x, y)


In [6]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248776 entries, 0 to 248775
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   attribute1  248776 non-null  int64
 1   attribute2  248776 non-null  int64
 2   attribute3  248776 non-null  int64
 3   attribute4  248776 non-null  int64
 4   attribute5  248776 non-null  int64
 5   attribute6  248776 non-null  int64
 6   attribute7  248776 non-null  int64
 7   attribute8  248776 non-null  int64
 8   attribute9  248776 non-null  int64
dtypes: int64(9)
memory usage: 17.1 MB


In [7]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 248776 entries, 0 to 248775
Series name: failure
Non-Null Count   Dtype
--------------   -----
248776 non-null  int64
dtypes: int64(1)
memory usage: 1.9 MB


# Tahminimizi yapıyoruz

In [8]:
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def algo_test(x, y):
    # Classification algorithms
    LR = LogisticRegression()
    ETR = ExtraTreeClassifier()
    GBR = GradientBoostingClassifier()
    kn = KNeighborsClassifier()
    dt = DecisionTreeClassifier()
    xgb = XGBClassifier()
    ab = AdaBoostClassifier()
    rf = RandomForestClassifier()

    algos = [LR, ETR, GBR, kn, dt, xgb, ab, rf]
    algo_names = ['Logistic Regression', 'Extra Tree', 'Gradient Boosting', 'KNeighbors', 'Decision Tree',
                   'XGBClassifier', 'AdaBoost', 'Random Forest']

    # Splitting into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    accuracy = []
    precision = []
    recall = []
    f1 = []

    result = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1_Score'], index=algo_names)

    for algo in algos:
        # Training the model and making predictions
        algo.fit(x_train, y_train)
        y_pred = algo.predict(x_test)

        # Calculating performance metrics
        accuracy.append(accuracy_score(y_test, y_pred))
        precision.append(precision_score(y_test, y_pred, average='weighted'))
        recall.append(recall_score(y_test, y_pred, average='weighted'))
        f1.append(f1_score(y_test, y_pred, average='weighted'))

    result['Accuracy'] = accuracy
    result['Precision'] = precision
    result['Recall'] = recall
    result['F1_Score'] = f1

    # Sorting performance results
    rtable = result.sort_values('F1_Score', ascending=False)
    return rtable

In [9]:
algo_test(x, y)

,Accuracy,Precision,Recall,F1_Score
Random Forest,0.999015,0.999016,0.999015,0.999015
Decision Tree,0.997488,0.997489,0.997488,0.997488
Extra Tree,0.997347,0.997350,0.997347,0.997347
XGBClassifier,0.996744,0.996752,0.996744,0.996744
Gradient Boosting,0.959100,0.959180,0.959100,0.959097
AdaBoost,0.939344,0.939652,0.939344,0.939338
KNeighbors,0.854570,0.858251,0.854570,0.854151
Logistic Regression,0.786398,0.823145,0.786398,0.780387
